In [3]:
# Import Python packages
import praw
from datetime import datetime
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import sys


In [75]:
# Load required variables from .env file.
load_dotenv(dotenv_path=Path("C:\\Python\\azure-openai-lab\\.venv\\.env")) #Error sometimes due to \ or \\. Try one or the other. "C:\\Python\\azure-openai-lab\\.venv\\.env"

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY_P34']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT_P34']

reddit_api_id = os.environ['REDDIT_API_ID']
reddit_api_secret = os.environ['REDDIT_API_SECRET']
reddit_api_user = os.environ['REDDIT_API_USER']

In [5]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_oai_endpoint, 
        api_key=azure_oai_key,  
        api_version="2024-10-21" #"2024-02-01"
        )

In [ ]:


# 1) Authenticate
reddit = praw.Reddit(
    client_id = reddit_api_id,
    client_secret = reddit_api_secret,
    user_agent = "CommentRetriever/0.1 by u/" + reddit_api_user
)

# 2) Fetch the submission
submission = reddit.submission(url="https://www.reddit.com/r/PowerBI/comments/1k2upuw/does_upgrading_to_fabric_capacity_workspace_make/")

# 4) Start building the output string with the post metadata and body
lines = []
post_time = datetime.utcfromtimestamp(submission.created_utc).isoformat()
lines.append(f"Post: {submission.title} (u/{submission.author})")
lines.append(f"Time: {post_time}")
lines.append(f"URL:  {submission.url}\n")
lines.append(submission.selftext or "[no body text]")
lines.append("\n---\nComments:")

# 5) Load all comments
submission.comments.replace_more(limit=None)

# 6) Flatten and append each comment to output
for comment in submission.comments.list():
    c_time = datetime.utcfromtimestamp(comment.created_utc).isoformat()
    author = comment.author or "[deleted]"
    body   = comment.body.replace('\n', ' ')
    lines.append(f"[{c_time}] u/{author}: {body}")

output = "\n".join(lines)

In [ ]:
# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as social media expert Dr. Rachna Jain who specializes in strategies for using psychological principles to enhance social media content.
Action: Create a social media post telling a story in the active voice that provide practical business value by Focusing on a single and clear Goal. The structure should be authentic and in the third person. The Text should be clear and professional (no emojis). Use comments that answer the question posed by the post and summarize these as solutions.
Target Audience: The recipients of the post are business leaders, managers and professionals who are curious about the topic.

### EXAMPLE

Title

Story-telling...

Implications...

1. Negative effect 1

2. Negative effect 2

3. Negative effect 3

Counteracting...

1. Solution 1

2. Solution 2

3. Solution 3
"""

In [74]:
# Zero-Shot learning. Model has a token limit of 4096.

# Send request to Azure OpenAI model
response = client.chat.completions.create(
    model="gpt-4.5-preview",       # or whichever model you prefer
    temperature=0.4,
    messages=[
        {
            "role": "system",
            "content": systemcontent
        },
        {
            "role": "user",
            "content": (
                "Create a post:\n"
                + "---\n"
                + output
            )
        }
    ]
)

result = response.choices[0].message.content
print(result + "\n")

Optimizing Data Storage with PowerBI: Is Fabric Capacity Right for Your Business?

Did you know that standard PowerBI Pro licenses have dataset size limitations that can restrict your ability to store extensive historical data?

Implications of staying with PowerBI Pro workspace:

1. Dataset size limitations may prevent long-term storage of sensor data from multiple plants.
2. Frequent deletion and reloading of tables consumes significant resources and reduces efficiency.
3. Potential loss of valuable historical insights due to limited data retention.

Counteracting these limitations effectively:

1. Consider upgrading to Fabric capacity workspace, allowing larger dataset storage and incremental refresh capabilities.
2. Evaluate your specific data requirements and choose a Fabric SKU that aligns with your storage and performance needs.
3. Implement incremental refresh strategies available in both Pro and Fabric capacities to optimize data management and reduce resource usage.

Communit